# Kata 1 - Lith Log

Agile provided a tutorial on interacting with their API to request problems and submit solutions. I'll use their code here.

In [1]:
import requests

url = 'https://kata.geosci.ai/challenge/sequence'  # <--- In week 2, you'll change the name.

r = requests.get(url)
#r.status_code

What comes back is "r" -- a request -- that has different attributes, including .status_code and .text.

.text is the description of the problem in Markdown. Let's have a look:

In [2]:
from IPython.display import Markdown

Markdown(r.text)

# Challenge: Sequence

You have a string of lithology codes, reading from the **bottom up** of a geological section. There is a sample every metre. There are three lithologies:

- **M**udstone
- **F**ine sandstone or siltstone
- **S**andstone

The strings look like this:

      ...MFFSSFSSSS...

Your data, when you receive it, will be much longer than this.

We need to get some geological information from this string of codes. Specifically, you need to answer 3 questions:

1. What is the total thickess in metres of sandstone (`S`)? Each sample represents one metre.
2. How many sandstone beds are there? A bed is a contiguous group of one lithology, so `MMFFF` is 2 beds, one of `M` and one of `F`.
3. How many times does the most common *upwards* bed transition occur? Do not include transitions from a lithology to itself.

Remember that the sequence is given to you from the bottom up. So an upwards transition is equivalent to a transition to the right.


## Example

Here is some example input:

      SSMMFFFFFFFFSSMFFSSFSSSSFMFSSSSFFSSFFFMM
      ^^          ^^   ^^ ^^^^   ^^^^  ^^

And the answers to the 3 questions:

- In this example, the total thickess of sandstone is 16 m. So the required answer is: **16**
- There are 6 sandstone beds in the sequence (marked above). The answer is: **6**
- The most common bed transition is `F` to `S`, which occurs 5 times. So the answer is: **5**


## A quick reminder how this works

You can retrieve your data by choosing any Python string as a **`<KEY>`** and substituting here:
    
    https://kata.geosci.ai/challenge/sequence?key=<KEY>
                                                  ^^^^^
                                                  use your own string here

To answer question 1, make a request like:

    https://kata.geosci.ai/challenge/sequence?key=<KEY>&question=1&answer=1234
                                                  ^^^^^          ^        ^^^^
                                                  your key       Q        your answer

[Complete instructions at kata.geosci.ai](https://kata.geosci.ai/challenge)

----

© 2020 Agile Scientific, licensed CC-BY

## Load my input

Okay, let's get an input for our problem. I'll use a "key":

In [3]:
my_key = "scibbatical"

params = {'key': my_key}

r = requests.get(url, params)

# The problem states that the input is 20 000 characters long. Look at the first ones:
r.text[:100]

'SSMMFMMMMMFFFFMMMMMMMFFFFFFFFMMFSSSFFFSSSSSSMFFMMMFFFFFFSSSSFFSSMMMMFFFSSMMMMFFFFFFFSFFMMMSSFMMSFSMS'

## Question 1

_What is the total thickess in metres of sandstone (S)? Each sample represents one metre._

Okay, this seems simple... just find the total number of S in the input, right?

In [4]:
log = r.text[:]

# Find the length of the string when all Ms and Fs are removed
answer1 = len(log.replace('M','').replace('F',''))

answer1

5535

Now, submit the answer using the method suggested by Agile:

In [10]:
params = {'key': my_key,   # <--- must be the same key as before
          'question': 1,   # <--- which question you're answering
          'answer': answer1,  # <--- your answer to that question
         }

r = requests.get(url, params)

r.text

'Correct'

Sweet as!

## Question 2

_How many sandstone beds are there? A bed is a contiguous group of one lithology, so MMFFF is 2 beds, one of M and one of F._

Interesting! Should I construct a simple counter? Use rules to determine the relationships between the intervals as the input string is read through like magnetic tape?

Or should I create a Bed class that has attributes like lith type, thickness, and associated lithology above and below?

In [5]:
class Bed:
    
    def __init__(self, lith, thickness, lith_below, lith_above):
        self.lith = lith
        self.thickness = thickness
        self.lith_below = lith_below
        self.lith_above = lith_above

Create a function that looks at a lithology log and creates the next bed.

In [6]:
def next_bed(log):
    # given a log, create a Bed, then update the return
    # the new (shorter) log
    # NOTE: the first character denotes the lithology of the
    # previous bed, so the second character is the first of the bed
    # that is created.
    
    global bed_idx
    
    lithlib = 'SFM'
    lith = log[1]
    
    # Find the next interface
    next_interface = min([log[1:].find(lithlib.replace(lith,'')[0]),
                          log[1:].find(lithlib.replace(lith,'')[1])])
    
    if next_interface == -1:
        next_interface = max([log[1:].find(lithlib.replace(lith,'')[0]),
                          log[1:].find(lithlib.replace(lith,'')[1])])
        
    if next_interface == -1:
        return log
    
    # Create a bed
    vars()['bed'+str(bed_idx)] = Bed(lith, next_interface,
                                     log[0], log[next_interface+1])
    bed_dict[bed_idx] = vars()['bed'+str(bed_idx)]
    
    bed_idx += 1
    
    return log[next_interface:]

I'm also going to start a dictionary to keep track of all the beds being created.

In [7]:
bed_dict = {}

In [8]:
bed_idx = 0

# Create the first bed, then clip the log to leave just a single log value for first bed)

# Define some variables
lithlib = 'SFM'
lith0 = log[0]

# Find the first interface
first_interface = min([log.find(lithlib.replace(lith0,'')[0]),
                      log.find(lithlib.replace(lith0,'')[1])])

# Create the first bed, then add that bed into bed_dict
vars()['bed'+str(bed_idx)] = Bed(lith0, first_interface, 'none', log[first_interface])
bed_dict[bed_idx] = vars()['bed'+str(bed_idx)]
bed_idx += 1

# Modify the log to turn the first bed into a single character - the
# format expected for the next_bed function
log_return = log[first_interface-1:]


# Parse the rest of the log into Beds

while log_return != log:
    log = log_return[:]
    log_return = next_bed(log)
    

# Create the last bed

vars()['bed'+str(bed_idx)] = Bed(log[1], len(log)-1, log[0], 'none')
bed_dict[bed_idx] = vars()['bed'+str(bed_idx)]

Okay, the log has been parsed into Bed objects. Let's find out how many of those objects are sandstone lithology:

In [9]:
ss_beds = 0

for bed in bed_dict:
    if bed_dict[bed].lith == 'S':
        ss_beds += 1

ss_beds

2291

Submit the answer:

In [110]:
params = {'key': my_key,
          'question': 2,
          'answer': ss_beds,
         }

r = requests.get(url, params)

r.text

'Correct'

Sweet as!

## Question 3

_How many times does the most common upwards bed transition occur? Do not include transitions from a lithology to itself._

We're already set up pretty well to find this!

In [10]:
# Define counters for the possible upward transitions:
SF = 0
SM = 0
FS = 0
FM = 0
MS = 0
MF = 0

# For each Bed, find it's upward transition,
# then increase the appropriate counter
for bed in bed_dict:
    transition = bed_dict[bed].lith + bed_dict[bed].lith_above
    
    # The final bed transitions into type "none"... that's not valid!
    if transition[1:] != 'none':
        vars()[transition] += 1
     
print('SF = ',SF,
      '\nSM = ', SM,
      '\nFS = ', FS,
      '\nFM = ', FM,
      '\nMS = ', MS,
      '\nMF = ', MF)
    

SF =  994 
SM =  1297 
FS =  1906 
FM =  1401 
MS =  384 
MF =  2313


Looks like MF is the transition that occurs most frequently.

Submit the answer for question 3:

In [111]:
params = {'key': my_key,   # <--- must be the same key as before
          'question': 3,   # <--- which question you're answering
          'answer': MF,  # <--- your answer to that question
         }

r = requests.get(url, params)

r.text

'Correct! The next challenge is: https://kata.geosci.ai/challenge/boreholes - good luck!'